In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from datetime import datetime
import random
import os
import sys
# Get the current directory.
current_dir = os.getcwd()
# Add the `src` folder to the PYTHONPATH.
sys.path.append(os.path.join(current_dir, "../src"))
from scraping_functions import *

In [2]:
collect_tournaments(2023)

Acapulco
Adelaide 1
Adelaide 2
Antwerp
Astana
Atlanta
Auckland
Australian Open
Banja Luka
Barcelona
Basel
Bastad
Beijing
Buenos Aires
Chengdu
Cincinnati
Cordoba
Dallas
Davis Cup Finals
Davis Cup Finals Group Stage
Davis Cup Qualifiers
Delray Beach
Doha
Dubai
Eastbourne
Estoril
Geneva
Gstaad
Halle
Hamburg
Houston
Indian Wells
Kitzbuhel
Laver Cup
London
Los Cabos
Lyon
Madrid
Mallorca
Marrakech
Marseille
Metz
Miami
Monte-Carlo
Montpellier
Munich
Newport
Next Gen Atp Finals
Nitto Atp Finals
Paris
Pune
Rio De Janeiro
Roland Garros
Rome
Rotterdam
Santiago
Shanghai
'S-Hertogenbosch
Stockholm
Stuttgart
Tel Aviv
Tokyo
Toronto
Umag
United Cup
Us Open
Vienna
Washington
Wimbledon
Winston-Salem
Zhuhai
Acapulco 807
Adelaide 1 2843
Adelaide 2 8998
Antwerp 7485
Astana 9410
Atlanta 6116
Auckland 301
Australian Open 580
Banja Luka 2841
Barcelona 425
Basel 328
Bastad 316
Beijing 747
Buenos Aires 506
Chengdu 7581
Cincinnati 422
Cordoba 9158
Dallas 424
Davis Cup Finals 8099
Davis Cup Finals Group Stage 809

ValueError: All arrays must be of the same length

In [ ]:
tournaments = pd.DataFrame({'Name' : [], 'Id' : [], 'Start Date' : [], 'End Date' : []})
for year in range(2023, 2000, -1):
    addition = collect_tournaments(year)
    tournaments = pd.concat([tournaments, addition], axis = 0)
    break

In [ ]:
tournaments['Id']=tournaments['Id'].astype('int64')
tournaments['Year'] = tournaments['Start Date'].dt.year
tournaments_df = tournaments.set_index(['Name', 'Year'])
tournaments_df = tournaments_df.sort_index()
tournaments_df.to_csv('tournaments')

In [ ]:
players_df = pd.DataFrame({'Name' : [], 'Id' : []})
i=0
for index in tournaments_df.index:
    addition = add_players(index)
    players_df = pd.concat([players_df, addition], axis = 0)
    print(i)
    i += 1
players_df = players_df.drop_duplicates().reset_index().drop('index', axis = 1).set_index('Name')
players_df

In [ ]:
matches_df = pd.DataFrame({'Player 1' : [], 'Player 2' : [], 'Winner' : [], 'Score' : [], 'Tournament Name' : []})
matches_df['Score'] = matches_df['Score'].astype('float64')
tournaments_df = pd.read_csv('../Data/tournaments.csv').set_index(['Name', 'Year'])
i=0
for index in tournaments_df.index:
    addition = collect_tourney_data(index, tournaments_df)
    matches_df = pd.concat([matches_df,addition], axis = 0)
    print(i)
    i+=1
matches_df = matches_df.reset_index().drop('index', axis = 1)
matches_df['Year'] = matches_df['Year'].astype('int64')

In [ ]:
new_matches = pd.read_csv('../Data/old_matches.csv').drop('Unnamed: 0', axis = 1)
new_matches

In [ ]:
matches_df['Score']=matches_df['Score'].astype('object')
matches_df


In [ ]:
player1, player2, winner, r1, r2= list(new_matches['Player 1']), list(new_matches['Player 2']), list(new_matches['Winner']), list(new_matches['Player 1 Ranking']), list(new_matches['Player 2 Ranking'])
for i in range(len(player1)):
    if player1[i] != winner[i]:
        print('switch')
        temp = player1[i]
        player1[i] = player2[i]
        player2[i] = temp

        temp1 = r1[i]
        r1[i] = r2[i]
        r2[i] = temp1
        
new_matches['Player 1'], new_matches['Player 2'], new_matches['Player 1 Ranking'], new_matches['Player 2 Ranking']= player1, player2,r1, r2

In [ ]:
new_matches

In [ ]:
matches_df

In [ ]:
final_df = matches_df.merge(new_matches, how = 'left', on = ['Player 1', 'Player 2', 'Winner', 'Tournament Name', 'Year'])
final_df = final_df.drop('Score_y', axis = 1).rename({'Score_x' : 'Score'}, axis = 1)

In [ ]:
final_df.head(10)

In [ ]:
final_df.to_csv('matches.csv')